# STUDENT NAME: YUSUF OYEBANJI


# STUDENT ID: W20027750 


# COURSE CODE: KF7032

I imported the pyspark to be able to use Spark Context. Pyspark is an interface for Apache Spark in python programming. The Spark is application programming interface (API) in Python that allows us to write Spark Application. After importing pyspark, I imported SparkContext and SparkConf from pyspark. The SparkContext represents the connection to the spark clusters which is used to create Resilient Distributed Dataset (RDD)  and broadcast variables on that cluster.

In [ ]:
import pyspark
from pyspark import SparkContext, SparkConf

conf=SparkConf().setMaster('local').setAppName('breathest');
spark_context=SparkContext(conf=conf)

I imported SparkSession in other to create a SparkSession in my program in PySpark, I need to use the builder pattern method builder() which is explained below. getOrCreate() method returns an already existing SparkSession; if it does not exist, it creates a new SparkSession and the appName() is used to set my application name.

The pyspark.sql.types class is a base class of all types in PySpark which defined in a package pyspark.sql.types.* and they are used to create DataFrame with a specific type.

The PySpark has a built-in standard Aggregate functions which define in DataFrame API, it comes in handy when I want to make use of aggregate operations on DataFrame columns. Aggregate functions operate on a group of rows and calculate a single return value for every group. The count,avg and sum are what I used from aggregate function in my program.

Lastly, the desc returns a sort expression based on the descending order of the given column name.

In [32]:
#importing SparkSession from pyspark.sql
from pyspark.sql import SparkSession
#importing all types from pyspark.sql.type
from pyspark.sql.types import *
#importing count, average and sum from pyspark.sql.function
from pyspark.sql.functions import count,avg,sum
#importing col and desc from pyspark.sql.function
from pyspark.sql.functions import col, desc


In [38]:
spark =SparkSession\
.builder\
.appName('Python Spark SQL')\
.config('spark.some.config.option','some-value')\
.getOrCreate()

spark_context=spark.sparkContext
spark_context


<SparkContext master=local appName=breathest>

In [39]:
!pwd

/home/notebookuser


This function parseline returns a key value pair or tuple (Reason, 1).
The parseline ignores all the data except the reason and the imposed value of 1 is used to facilitate the count. It is possible to include all the remaining fields in the line as the return value of parseline. The lines variable is used to browse the file directory of the text file named "DigitalBreathTestData2013.txt" using spark_context variable that was created earlier with textFile. The Read a texFile from Hadoop Distributed File System ( HDFS ), a local file system (available on all nodes) or any Hadoop-supported file system URI, and return it as an Resilient Distributed Dataset (RDD) of Strings. 

In [40]:
def parseline(line):
    #The split() method splits a string into a list.
    fields = line.split(',')
    Reason =fields[0]
    return(Reason, 1)
lines=spark_context.textFile('DigitalBreathTestData2013.txt')
#lines=spark_context.textFile('Users/yusufoyebanji/Downloads/DigitalBreathTestData2013.txt')
lines.count()

497791

In [41]:
#The map is used to iterate all the fields. map() function returns a map object
#of the results after applying the given function to each item of a given iterable
allReasons=lines.map(parseline)

#extract header
header=lines.first()
justdata=lines.filter(lambda row:row !=header)


The parseline2 is used to get all the fields in the "DigitalBreathTestData2013.txt" and the return the fields. 


In [42]:
def parseline2(line):
    fields = line.split(',')
    Reason = fields[0]
    Month = fields [1]
    Year = fields [2]
    WeekType = fields [3]
    TimeBand = fields [4]
    BreathAlcoholLevel = int(fields [5])
    AgeBand = fields [6]
    
    if fields [7] == 'Female':
        Gender = 2
    else:
        Gender = 1
    return (Reason,Month,Year,WeekType,TimeBand,BreathAlcoholLevel,AgeBand,Gender)
        
    

PySpark StructType & StructField classes are used to specify the schema to the DataFrame and creating complex columns like nested struct, array and map columns. StructType is a collection of StructField’s that defines column name, column data type, boolean to specify if the field can be nullable or not and metadata. The StringType() Converts an internal SQL object into a native Python object.

In [43]:
myschema2 = StructType(
    [StructField('Reason', StringType(),True),
     StructField('Month', StringType(),True),
     StructField('Year', IntegerType(),True),
     StructField('WeekType', StringType(),True),
     StructField('TimeBand', StringType(),True),
     StructField('AlcoholLevel', IntegerType(),True),
     StructField('AgeBand', StringType(),True),
     StructField('Gender', StringType(),True)
     
    
    ]
)

The spark.read.load stored in the data_frame variable loads the text file and format it in Comma Separated Values (CSV). A CSV is a plain text file that contains a list of data. The comma (,) used in sep is used to separate the values and ".withColumnRenamed" returns a new DataFrame by renaming an existing column. This is a no-op if schema doesn’t contain the given column name.

In [44]:
data_frame = spark.read.load("DigitalBreathTestData2013.txt",
                             format = 'csv', sep = ',', 
                             schema = myschema2,
                             header= 'true')\
                            .withColumnRenamed('BreathAlcoholLevel(microg 100ml)', 'AlcoholLevel')

#to display the data frame
data_frame.show()

+--------------------+-----+----+--------+--------+------------+-------+------+
|              Reason|Month|Year|WeekType|TimeBand|AlcoholLevel|AgeBand|Gender|
+--------------------+-----+----+--------+--------+------------+-------+------+
|Moving Traffic Vi...|  Jan|2013| Weekday|12am-4am|          80|  30-39|  Male|
|Road Traffic Coll...|  Jan|2013| Weekday|12am-4am|           0|  Other|  Male|
|Road Traffic Coll...|  Jan|2013| Weekday|12am-4am|          96|  Other|  Male|
|Moving Traffic Vi...|  Jan|2013| Weekday|12am-4am|           0|  40-49|Female|
|Suspicion of Alcohol|  Jan|2013| Weekday|12am-4am|           0|  40-49|  Male|
|Road Traffic Coll...|  Jan|2013| Weekday|8am-12pm|          45|  Other|  Male|
|Moving Traffic Vi...|  Jan|2013| Weekday|12am-4am|          60|  30-39|  Male|
|Moving Traffic Vi...|  Jan|2013| Weekday|12am-4am|           0|  16-19|  Male|
|Moving Traffic Vi...|  Jan|2013| Weekday|12am-4am|           0|  16-19|  Male|
|Moving Traffic Vi...|  Jan|2013| Weekda

In [45]:
data_frame.schema

StructType(List(StructField(Reason,StringType,true),StructField(Month,StringType,true),StructField(Year,IntegerType,true),StructField(WeekType,StringType,true),StructField(TimeBand,StringType,true),StructField(AlcoholLevel,IntegerType,true),StructField(AgeBand,StringType,true),StructField(Gender,StringType,true)))

In [46]:
#The count return the number of times the values of Reason appears 
data_frame. agg(count('Reason')).show()

+-------------+
|count(Reason)|
+-------------+
|       497790|
+-------------+



In [47]:
#This select from the "Reason" where the AlcoholLevel is greater than 35 and then show the result after counting.
data_frame.select('Reason').where(data_frame.AlcoholLevel > 35).agg(count('Reason')).show()

+-------------+
|count(Reason)|
+-------------+
|        51488|
+-------------+



In [48]:
#This select ALL where the AlcoholLevel is greater than 35 and then group it by the 'Gender', 'AgeBand', 'Month'
#count the values based on the fields displayed. The "Reason" is changed to "Aggregate" by using the alias. The order 
#is used to sort the table using the "Aggregate" field and descending is set to false to start counting from the
#highest. The default is ascending if not set.
count_df = (data_frame.select('*')
        .where(data_frame.AlcoholLevel>35)
        .groupBy('Gender', 'AgeBand', 'Month')
        .agg(count("Reason").alias('Aggregate'))
        .orderBy('Aggregate', ascending=False)
               )

count_df.show()

+------+-------+-----+---------+
|Gender|AgeBand|Month|Aggregate|
+------+-------+-----+---------+
|  Male|  30-39|  Jun|      980|
|  Male|  30-39|  May|      868|
|  Male|  30-39|  Dec|      846|
|  Male|  Other|  Dec|      823|
|  Male|  30-39|  Jul|      805|
|  Male|  30-39|  Aug|      800|
|  Male|  30-39|  Mar|      782|
|  Male|  30-39|  Apr|      767|
|  Male|  Other|  Aug|      734|
|  Male|  30-39|  Oct|      732|
|  Male|  30-39|  Nov|      726|
|  Male|  40-49|  Jun|      698|
|  Male|  40-49|  May|      681|
|  Male|  20-24|  Jun|      653|
|  Male|  40-49|  Jul|      643|
|  Male|  20-24|  Dec|      639|
|  Male|  20-24|  May|      630|
|  Male|  20-24|  Mar|      627|
|  Male|  30-39|  Sep|      626|
|  Male|  25-29|  Jun|      625|
+------+-------+-----+---------+
only showing top 20 rows



In [50]:
#showing one single column which displays the largest target group  
count_df.show(1)

+------+-------+-----+---------+
|Gender|AgeBand|Month|Aggregate|
+------+-------+-----+---------+
|  Male|  30-39|  Jun|      980|
+------+-------+-----+---------+
only showing top 1 row

